**3. Do seasons affect average price of a listing?**<br>

Null Hypothesis: Seasons do not affect the price (x_mean - y_mean = 0)<br>
Alternate Hypothesis : Seasons affect the price (x_mean - y_mean != 0)<br>

We process the data to analyze below cases<br>
1. Summer(June,July,August) vs winter(Dec,Jan,Feb)
2. Spring(Sep,Oct,Nov) vs Fall(Mar,Apr,May)


In [32]:
import os
import csv
import glob
import pandas as pd
import numpy as np
from collections import Counter
import matplotlib.pyplot as plt
from scipy.stats import norm

In [33]:
months_summer = ['6','7','8']
months_winter = ['12','1','2']
months_fall = ['3','4','5']
months_spring = ['9','10','11']
summerdf = pd.DataFrame()
winterdf = pd.DataFrame()
falldf = pd.DataFrame()
springdf = pd.DataFrame()
boroughs = ['Manhattan','Bronx','Staten Island','Queens','Brooklyn']
for year in (2015,2016,2017):
    path = "data2/"+str(year)
    if os.path.exists(path):
        cols = ['id','price','neighbourhood_group_cleansed']
        allFiles = glob.glob(os.path.join(path,"*.csv"))
        
        for file_ in allFiles:
            iter_csv = pd.read_csv(file_, iterator=True, chunksize=1000, usecols = cols)
            df = pd.concat([chunk[chunk['neighbourhood_group_cleansed'].isin(boroughs)] for chunk in iter_csv])
            listings_count = len(df.index)
            
            #preprocess price
            df = df[df.price != 0]
            df['price'] = df['price'].replace('[\$,)]','',regex=True).replace('[(]','-', regex=True).astype(float)
            df = df.dropna(axis=0)
            
            #get month
            filename = file_.split('.')[0]
            month = filename.split('_')[1]
            
            #get winter vs summer months data
            if month in months_summer:
                if len(summerdf.index) == 0:
                    summerdf = pd.concat([df['price'], df['id']], axis=1)
                    summerdf['year'] = year
                else:
                    tempdf = pd.concat([df['price'], df['id']], axis=1)
                    tempdf['year'] = year
                    summerdf = pd.concat([summerdf,tempdf])
            elif month in months_winter:
                if len(winterdf.index) == 0:
                    winterdf = pd.concat([df['price'], df['id']], axis=1)
                    winterdf['year'] = year
                else:
                    tempdf = pd.concat([df['price'], df['id']], axis=1)
                    tempdf['year'] = year
                    winterdf = pd.concat([winterdf,tempdf])
            elif month in months_fall:
                if len(falldf.index) == 0:
                    falldf = pd.concat([df['price'], df['id']], axis=1)
                    falldf['year'] = year
                else:
                    tempdf = pd.concat([df['price'], df['id']], axis=1)
                    tempdf['year'] = year
                    falldf = pd.concat([falldf,tempdf])
            elif month in months_spring:
                if len(springdf.index) == 0:
                    springdf = pd.concat([df['price'], df['id']], axis=1)
                    springdf['year'] = year
                else:
                    tempdf = pd.concat([df['price'], df['id']], axis=1)
                    tempdf['year'] = year
                    springdf = pd.concat([springdf,tempdf])
            
            

**2. Summer vs winter<br>**
Data Pre-processing: <br>
We have summerdf and winterdf dataframes which have rows of the form [id,year,price]. We then take a join of the 2 dataframes on id and year to compare prices in the same year

In [34]:

summerdf = summerdf.rename(index=str, columns={"price": "summerprice", "id": "id","year":"year"})
summerdf = summerdf.groupby(['id','year'])['summerprice'].mean()
summerdf = summerdf.reset_index()

winterdf = winterdf.rename(index=str, columns={"price": "winterprice", "id": "id","year":"year"})
winterdf = winterdf.groupby(['id','year'])['winterprice'].mean()
winterdf = winterdf.reset_index()

falldf = falldf.rename(index=str, columns={"price": "fallprice", "id": "id","year":"year"})
falldf = falldf.groupby(['id','year'])['fallprice'].mean()
falldf = falldf.reset_index()

springdf = springdf.rename(index=str, columns={"price": "springprice", "id": "id","year":"year"})
springdf = springdf.groupby(['id','year'])['springprice'].mean()
springdf = springdf.reset_index()

#summer vs winter price
sum_win_result = pd.merge(winterdf,
                 summerdf[['summerprice','id','year']],
                 on=['id','year'])

win_sum_price = sum_win_result['winterprice'].tolist()
sum_win_price = sum_win_result['summerprice'].tolist()
listings_count = len(sum_win_result.index)
print "Total common listings in summer and winter: " + str(listings_count)
count = sum_win_result[sum_win_result['winterprice'] > sum_win_result['summerprice']]
print "No of listings that have prices greater in winter: "+ str(len(count.index))
count = sum_win_result[sum_win_result['summerprice'] > sum_win_result['winterprice']]
print "No of listings that have prices greater in summer: "+ str(len(count.index))
count = sum_win_result[sum_win_result['summerprice'] == sum_win_result['winterprice']]
print "No of listings that have prices same in summer and winter: "+ str(len(count.index))

#summer vs fall price
sum_fall_result = pd.merge(falldf,
                 summerdf[['summerprice','id','year']],
                 on=['id','year'])

fall_sum_price = sum_fall_result['fallprice'].tolist()
sum_fall_price = sum_fall_result['summerprice'].tolist()
listings_count = len(sum_fall_result.index)
print "Total common listings in summer and fall: " + str(listings_count)
count = sum_fall_result[sum_fall_result['fallprice'] > sum_fall_result['summerprice']]
print "No of listings that have prices greater in fall: "+ str(len(count.index))
count = sum_fall_result[sum_fall_result['summerprice'] > sum_fall_result['fallprice']]
print "No of listings that have prices greater in summer: "+ str(len(count.index))
count = sum_fall_result[sum_fall_result['summerprice'] == sum_fall_result['fallprice']]
print "No of listings that have prices same in summer and fall: "+ str(len(count.index))

#summer vs spring price
sum_spr_result = pd.merge(springdf,
                 summerdf[['summerprice','id','year']],
                 on=['id','year'])

spr_sum_price = sum_spr_result['springprice'].tolist()
sum_spr_price = sum_spr_result['summerprice'].tolist()
listings_count = len(sum_spr_result.index)
print "Total common listings in summer and spring: " + str(listings_count)
count = sum_spr_result[sum_spr_result['springprice'] > sum_spr_result['summerprice']]
print "No of listings that have prices greater in spring: "+ str(len(count.index))
count = sum_spr_result[sum_spr_result['summerprice'] > sum_spr_result['springprice']]
print "No of listings that have prices greater in summer: "+ str(len(count.index))
count = sum_spr_result[sum_spr_result['summerprice'] == sum_spr_result['springprice']]
print "No of listings that have prices same in summer and spring: "+ str(len(count.index))

#winter vs fall price
win_fall_result = pd.merge(winterdf,
                 falldf[['fallprice','id','year']],
                 on=['id','year'])

win_fall_price = win_fall_result['winterprice'].tolist()
fall_win_price = win_fall_result['fallprice'].tolist()
listings_count = len(win_fall_result.index)
print "Total common listings in winter and fall: " + str(listings_count)
count = win_fall_result[win_fall_result['winterprice'] > win_fall_result['fallprice']]
print "No of listings that have prices greater in winter: "+ str(len(count.index))
count = win_fall_result[win_fall_result['fallprice'] > win_fall_result['winterprice']]
print "No of listings that have prices greater in fall: "+ str(len(count.index))
count = win_fall_result[win_fall_result['fallprice'] == win_fall_result['winterprice']]
print "No of listings that have prices same in winter and fall: "+ str(len(count.index))

#winter vs spring price
win_spr_result = pd.merge(springdf,
                 winterdf[['winterprice','id','year']],
                 on=['id','year'])

spr_win_price = win_spr_result['springprice'].tolist()
win_spr_price = win_spr_result['winterprice'].tolist()
listings_count = len(sum_spr_result.index)
print "Total common listings in winter and spring: " + str(listings_count)
count = win_spr_result[win_spr_result['springprice'] > win_spr_result['winterprice']]
print "No of listings that have prices greater in spring: "+ str(len(count.index))
count = win_spr_result[win_spr_result['winterprice'] > win_spr_result['springprice']]
print "No of listings that have prices greater in winter: "+ str(len(count.index))
count = win_spr_result[win_spr_result['winterprice'] == win_spr_result['springprice']]
print "No of listings that have prices same in winter and spring: "+ str(len(count.index))

#fall vs spring price
fall_spr_result = pd.merge(springdf,
                 falldf[['fallprice','id','year']],
                 on=['id','year'])

spr_fall_price = fall_spr_result['springprice'].tolist()
fall_spr_price = fall_spr_result['fallprice'].tolist()
listings_count = len(sum_spr_result.index)
print "Total common listings in fall and spring: " + str(listings_count)
count = fall_spr_result[fall_spr_result['springprice'] > fall_spr_result['fallprice']]
print "No of listings that have prices greater in spring: "+ str(len(count.index))
count = fall_spr_result[fall_spr_result['fallprice'] > fall_spr_result['springprice']]
print "No of listings that have prices greater in fall: "+ str(len(count.index))
count = fall_spr_result[fall_spr_result['fallprice'] == fall_spr_result['springprice']]
print "No of listings that have prices same in fall and spring: "+ str(len(count.index))





Total common listings in summer and winter: 104013
No of listings that have prices greater in winter: 19860
No of listings that have prices greater in summer: 20661
No of listings that have prices same in summer and winter: 63492
Total common listings in summer and fall: 94718
No of listings that have prices greater in fall: 10968
No of listings that have prices greater in summer: 17774
No of listings that have prices same in summer and fall: 65976
Total common listings in summer and spring: 100157
No of listings that have prices greater in spring: 16215
No of listings that have prices greater in summer: 12627
No of listings that have prices same in summer and spring: 71315
Total common listings in winter and fall: 92478
No of listings that have prices greater in winter: 19691
No of listings that have prices greater in fall: 15593
No of listings that have prices same in winter and fall: 57194
Total common listings in winter and spring: 100157
No of listings that have prices greater in 

** 2 population wald's test**<br>
Walds statistic for comparison of population of 2 means, the p-value and confidence interval are displayed.The significance level alpha = 0.05

In [35]:
# 2 population wald's test
def twopop_walds(X,Y):
    

    #calculate variances
    x_var = np.var(X)
    y_var = np.var(Y)
    #print x_var
    
    #calculate means
    x_mean = np.mean(X)
    y_mean = np.mean(Y)
    diff = x_mean - y_mean

    se_hat = np.sqrt(x_var/len(X) + y_var/len(Y))
    #print se_hat

    w = np.divide((diff),se_hat)
    print "walds statistic: "+str(w)
    
    z_critical = norm.ppf(0.975)
    margin_of_error = z_critical * se_hat
    confidence_interval = (diff - margin_of_error, diff + margin_of_error)
    
    pVal = 2*norm.cdf(-abs(w))
    
    print "p-value: " + str(pVal)
    print "CI: " + str(confidence_interval)

    if abs(w)>1.96:
        print("Null Hypothesis Rejected")
    else:
        print("Null Hypothesis Accepted")

Compute the 2 populate walds test for the 3 cases mentioned above

In [36]:
#summer vs winter price
A = np.array(sum_win_price,dtype=np.float32)
B = np.array(win_sum_price,dtype=np.float32)
twopop_walds(A,B)

#summer vs fall price
A = np.array(sum_fall_price,dtype=np.float32)
B = np.array(fall_sum_price,dtype=np.float32)
twopop_walds(A,B)

#summer vs spring price
A = np.array(sum_spr_price,dtype=np.float32)
B = np.array(spr_sum_price,dtype=np.float32)
twopop_walds(A,B)

#winter vs fall price
A = np.array(win_fall_price,dtype=np.float32)
B = np.array(fall_win_price,dtype=np.float32)
twopop_walds(A,B)

#winter vs spring price
A = np.array(win_spr_price,dtype=np.float32)
B = np.array(spr_win_price,dtype=np.float32)
twopop_walds(A,B)

#fall vs spring price
A = np.array(fall_spr_price,dtype=np.float32)
B = np.array(spr_fall_price,dtype=np.float32)
twopop_walds(A,B)




walds statistic: 0.2763550610842777
p-value: 0.78227536918332
CI: (-1.4557460604648993, 1.9336513339023993)
Null Hypothesis Accepted
walds statistic: 1.6934891771452436
p-value: 0.09036239322129748
CI: (-0.25378425782943737, 3.4794617480638124)
Null Hypothesis Accepted
walds statistic: -1.3893125857981405
p-value: 0.16473771839440432
CI: (-3.065039027749667, 0.5222228656402923)
Null Hypothesis Accepted
walds statistic: 1.1905190948913502
p-value: 0.23384242948402945
CI: (-0.693884288968128, 2.8411010858431283)
Null Hypothesis Accepted
walds statistic: -1.8702467945466255
p-value: 0.0614495542410309
CI: (-3.1861107850001638, 0.07463007210953876)
Null Hypothesis Accepted
walds statistic: -2.83921069664491
p-value: 0.004522528395941694
CI: (-5.099795780181902, -0.934322872161848)
Null Hypothesis Rejected


**Conclusion**<br>
1. At 5% level of significance, the data provides sufficient evidence that the average price of a listing doe not vary during comic con when compared to prev non event months and all non event months<br>